In [1]:
import pandas as pd
left_data=pd.read_excel('最早数据.xlsx',converters={'证券代码':str})
right_data=pd.read_csv('股票列表.csv',converters={'symbol':str})
data=pd.merge(left=left_data,right=right_data,how='inner',left_on='证券代码',right_on='symbol')

In [2]:
import tushare as ts
pro = ts.pro_api()
code=data['ts_code'].tolist()
for i in code:
    df = pro.daily_basic(ts_code=str(i),start_date='20210301',end_date='20210601')
    df.to_csv('每日指标数据/'+str(i[0:6])+'.csv',index=False)

In [7]:
from  sklearn import ensemble
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
def set_missing_ages(df):
	#把数值型特征都放到随机森林里面去
    fill_list=['close','turnover_rate','turnover_rate_f','volume_ratio',
               'pe','pe_ttm','pb','ps','ps_ttm','dv_ratio','dv_ttm',
               'total_share','float_share','free_share','total_mv','circ_mv']
    age_df=df[['close','turnover_rate','turnover_rate_f','volume_ratio',
               'pe','pe_ttm','pb','ps','ps_ttm','dv_ratio','dv_ttm',
               'total_share','float_share','free_share','total_mv','circ_mv']]
    for feature in fill_list:
        if age_df[feature].count()==age_df.shape[0]:
            continue
        temp = age_df[feature]
        age_df = age_df.drop(feature,axis=1)
        age_df.insert(0,feature,temp)
        known_age = age_df[age_df[feature].notnull()].fillna(value=0)
        unknown_age = age_df[age_df[feature].isnull()].fillna(value=0)
        y=known_age.iloc[:,0]#y是第一列数据
        if y.shape[0]==0:
            print(feature)
            continue
        x=known_age.iloc[:,1:]#x是特征属性值，后面几列
        rfr=tree.DecisionTreeRegressor()
        #根据已有数据去拟合随机森林模型
        rfr.fit(x,y)
    #预测缺失值
        predictedAges = rfr.predict(unknown_age.iloc[:,1:])
    #填补缺失值
        df.loc[(df[feature].isnull()),feature] = predictedAges

    return df

In [10]:
import os
files=os.listdir('每日指标数据')
for i in range(0,len(files)):
    temp=pd.read_csv("每日指标数据/"+str(files[i]))
    temp=set_missing_ages(temp)
    temp=temp.drop(columns=['pe', 'pe_ttm','dv_ratio','dv_ttm'])
    temp.to_csv('填补完成/'+str(files[i]),index=False)

pe
pe_ttm
dv_ratio
dv_ttm
dv_ratio
dv_ttm
dv_ratio
dv_ttm
pe_ttm
dv_ratio
dv_ttm
dv_ratio
dv_ttm
dv_ratio
dv_ttm
pe_ttm
dv_ratio
dv_ratio
dv_ttm


In [13]:
import os
files=os.listdir('填补完成')
data=pd.read_csv('填补完成/'+str(files[0]))
for i in range(1,len(files)):
    temp_data=pd.read_csv('填补完成/'+str(files[i]))
    data=pd.concat([data,temp_data])
data=data.drop(columns=['turnover_rate_f','ps_ttm','total_share','float_share',
                        'free_share','circ_mv'])

In [40]:
date=list(set(data['trade_date'].tolist()))
date.sort()
result=[]
for i in date:
    temp_date=data[data['trade_date']==i]
    total_mv=sum(temp_date['total_mv'].tolist())
    temp_date['close']=temp_date['close']*temp_date['total_mv']/total_mv
    temp_date['turnover_rate']=temp_date['turnover_rate']*temp_date['total_mv']/total_mv
    temp_date['volume_ratio']=temp_date['volume_ratio']*temp_date['total_mv']/total_mv
    temp_date['pb']=temp_date['pb']*temp_date['total_mv']/total_mv
    temp_date['ps']=temp_date['ps']*temp_date['total_mv']/total_mv
    temp_close=sum(temp_date['close'].tolist())
    temp_turnover_rate=sum(temp_date['turnover_rate'].tolist())
    temp_volume_ratio=sum(temp_date['volume_ratio'].tolist())
    temp_pb=sum(temp_date['pb'].tolist())
    temp_ps=sum(temp_date['ps'].tolist())
    result.append([i,temp_close,temp_turnover_rate,temp_volume_ratio,
                   temp_pb,temp_ps])
result=pd.DataFrame(result)
result.columns = ['date', 'close', 'turnover_rate', 'volume_ratio',
                  'pb','ps']

<ipython-input-40-8bd0d8aa99db>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_date['close']=temp_date['close']*temp_date['total_mv']/total_mv
<ipython-input-40-8bd0d8aa99db>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_date['turnover_rate']=temp_date['turnover_rate']*temp_date['total_mv']/total_mv
<ipython-input-40-8bd0d8aa99db>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [42]:
result.to_csv('index_indicator.csv',index=False)